In [10]:
import sys
import os
sys.path.append('../')

import pandas as pd

from models import VARModel, ARDLModel
import data_opener

import baselines.bivariate_granger as bg
from baselines.tft import TFT_baseline
import baselines.rfe as rfe

import numpy as np
import networkx as nx
from cdt.metrics import SHD
import matplotlib.pyplot as plt
import time

random_generator = np.random.default_rng(0)

In [11]:
def separate_train_test(df):
    df_train = df.iloc[:int(0.7*len(df))]
    df_test = df.iloc[int(0.7*len(df)):]
    return df_train, df_test

# First baseline : BG, with ARDL model evaluator

In [3]:
rootdir = "../"
dataset = "SynthNonlin/7ts2h"


maxlags = 10
model_config = {
    "constructor":{
            "lags": maxlags,
            "order": maxlags,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

df_results = []

for filename in os.listdir(rootdir + "data/" + dataset + "/"):
    if not os.path.isfile(rootdir + "data/" + dataset + "/" + filename):
        continue
    print("File",filename,"has begun.")
    
    df, _, causes = data_opener.open_dataset_and_ground_truth(dataset, filename, rootdir=rootdir)
        
    df_train, df_test = separate_train_test(df)
    
    targets = df.columns
    
    for target in targets:
        if len(causes[target]) == 0:
            continue
        else:
            print("\tVariable", target, "is beginning.")
            
            time_algostart = time.time()
            lPred = bg.bivariate_granger(df_train, target, maxlags)
            time_algoend = time.time()
            
            #add autoregressive cause as the algorithm does not return it
            if target not in lPred:
                lPred.append(target)
            
            sPred = set(lPred)
            sTrue = set(causes[target])

            # metrics from results
            sTP = sTrue & sPred
            recall = len(sTP)/len(sTrue)
            precision = len(sTP)/len(sPred)
            
            base_row = {"dataset":dataset,"filename":filename,'target':target,
                        "totaltime":time_algoend - time_algostart,
                        "final_precision":precision,"final_recall":recall,
                        "final_TP":len(sTP), 
                        "final_FP":len(sPred)-len(sTP),
                        "final_FN": len(sTrue)-len(sTP), 
                        "final_TN": len(df.columns)+len(sTP)-len(sPred)-len(sTrue)}
            
            model = ARDLModel(model_config, target)
            df_test_bis = df_test.reset_index()  # Statsmodels has uncomprehensible behaviour
            df_train_bis = df_train.reset_index()  # Statsmodels has uncomprehensible behaviour
            model.fit(df_train_bis[lPred])
            test_statistics = model.statistics(df_test_bis[lPred])
            test_statistics = dict([("test_"+x,test_statistics[x]) for x in test_statistics])
            
            base_row = {**base_row, **test_statistics}
            
            df_results.append(base_row)
            
df = pd.DataFrame(df_results)
df["baseline"]="bg"
df["model"]="ARDL"

df.to_csv("baselines.csv",index=False)

File data_1.csv has begun.
	Variable E is beginning.
	Variable B is beginning.
	Variable F is beginning.
	Variable C is beginning.
	Variable H is beginning.
	Variable D is beginning.
	Variable A is beginning.
File data_0.csv has begun.
	Variable E is beginning.
	Variable B is beginning.
	Variable F is beginning.
	Variable C is beginning.
	Variable H is beginning.
	Variable D is beginning.
	Variable A is beginning.
File data_2.csv has begun.
	Variable E is beginning.
	Variable B is beginning.
	Variable F is beginning.
	Variable C is beginning.
	Variable H is beginning.
	Variable D is beginning.
	Variable A is beginning.
File data_6.csv has begun.
	Variable E is beginning.
	Variable B is beginning.
	Variable F is beginning.
	Variable C is beginning.
	Variable H is beginning.
	Variable D is beginning.
	Variable A is beginning.
File data_9.csv has begun.
	Variable E is beginning.
	Variable B is beginning.
	Variable F is beginning.
	Variable C is beginning.
	Variable H is beginning.
	Variab

In [4]:
rootdir = "../"
dataset = "fMRI_processed_by_Nauta/returns/our_selection"

maxlags = 5
model_config = {
    "constructor":{
            "lags": maxlags,
            "order": maxlags,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

df_results = []

for filename in os.listdir(rootdir + "data/" + dataset + "/"):
    if not os.path.isfile(rootdir + "data/" + dataset + "/" + filename):
        continue
    print("File",filename,"has begun.")
    
    df, _, causes = data_opener.open_dataset_and_ground_truth(dataset, filename, rootdir=rootdir)
        
    df_train, df_test = separate_train_test(df)
    
    targets = df.columns
    
    for target in targets:
        if len(causes[target]) == 0:
            continue
        else:
            print("\tVariable", target, "is beginning.")
            
            time_algostart = time.time()
            lPred = bg.bivariate_granger(df_train, target, maxlags)
            time_algoend = time.time()
            
            #add autoregressive cause as the algorithm does not return it
            if target not in lPred:
                lPred.append(target)
            
            sPred = set(lPred)
            sTrue = set(causes[target])

            # metrics from results
            sTP = sTrue & sPred
            recall = len(sTP)/len(sTrue)
            precision = len(sTP)/len(sPred)
            
            base_row = {"dataset":dataset,"filename":filename,'target':target,
                        "totaltime":time_algoend - time_algostart,
                        "final_precision":precision,"final_recall":recall,
                        "final_TP":len(sTP), 
                        "final_FP":len(sPred)-len(sTP),
                        "final_FN": len(sTrue)-len(sTP), 
                        "final_TN": len(df.columns)+len(sTP)-len(sPred)-len(sTrue)}
            
            model = ARDLModel(model_config, target)
            df_test_bis = df_test.reset_index()  # Statsmodels has uncomprehensible behaviour
            df_train_bis = df_train.reset_index()  # Statsmodels has uncomprehensible behaviour
            model.fit(df_train_bis[lPred])
            test_statistics = model.statistics(df_test_bis[lPred])
            test_statistics = dict([("test_"+x,test_statistics[x]) for x in test_statistics])
            
            base_row = {**base_row, **test_statistics}
            
            df_results.append(base_row)
            
df = pd.DataFrame(df_results)
df["baseline"]="bg"
df["model"]="ARDL"

df.to_csv("baselines.csv",mode="a", index=False, header=False)

File timeseries21.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries7.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries23.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries15.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries16.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries18.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries8.csv ha

In [5]:
rootdir = "../"
dataset = "TestCLIM_N-5_T-250/returns"

maxlags = 5
model_config = {
    "constructor":{
            "lags": maxlags,
            "order": maxlags,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

df_results = []

for filename in os.listdir(rootdir + "data/" + dataset + "/"):
    if not os.path.isfile(rootdir + "data/" + dataset + "/" + filename):
        continue
    print("File",filename,"has begun.")
    
    df, _, causes = data_opener.open_dataset_and_ground_truth(dataset, filename, rootdir=rootdir)
        
    df_train, df_test = separate_train_test(df)
    
    targets = df.columns
    
    for target in targets:
        if len(causes[target]) == 0:
            continue
        else:
            print("\tVariable", target, "is beginning.")
            
            time_algostart = time.time()
            lPred = bg.bivariate_granger(df_train, target, maxlags)
            time_algoend = time.time()
            
            #add autoregressive cause as the algorithm does not return it
            if target not in lPred:
                lPred.append(target)
            
            sPred = set(lPred)
            sTrue = set(causes[target])

            # metrics from results
            sTP = sTrue & sPred
            recall = len(sTP)/len(sTrue)
            precision = len(sTP)/len(sPred)
            
            base_row = {"dataset":dataset,"filename":filename,'target':target,
                        "totaltime":time_algoend - time_algostart,
                        "final_precision":precision,"final_recall":recall,
                        "final_TP":len(sTP), 
                        "final_FP":len(sPred)-len(sTP),
                        "final_FN": len(sTrue)-len(sTP), 
                        "final_TN": len(df.columns)+len(sTP)-len(sPred)-len(sTrue)}
            
            model = ARDLModel(model_config, target)
            df_test_bis = df_test.reset_index()  # Statsmodels has uncomprehensible behaviour
            df_train_bis = df_train.reset_index()  # Statsmodels has uncomprehensible behaviour
            model.fit(df_train_bis[lPred])
            test_statistics = model.statistics(df_test_bis[lPred])
            test_statistics = dict([("test_"+x,test_statistics[x]) for x in test_statistics])
            
            base_row = {**base_row, **test_statistics}
            
            df_results.append(base_row)
            
df = pd.DataFrame(df_results)
df["baseline"]="bg"
df["model"]="ARDL"

df.to_csv("baselines.csv",mode="a", index=False, header=False)

File TestCLIM_N-5_T-250_0089.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0074.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0149.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0055.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0154.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0110.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variabl

	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0091.txt has begun.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0072.txt has begun.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0075.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0008.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0081.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0052.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0006.txt h

	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0169.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0167.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0080.txt has begun.
	Variable 0 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0077.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0078.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0073.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variabl

	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0130.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0046.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0005.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0178.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0064.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_01

In [6]:
rootdir = "../"
dataset = "FinanceCPT/returns/our_selection"

maxlags = 10
model_config = {
    "constructor":{
            "lags": maxlags,
            "order": maxlags,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

df_results = []

for filename in os.listdir(rootdir + "data/" + dataset + "/"):
    if not os.path.isfile(rootdir + "data/" + dataset + "/" + filename):
        continue
    print("File",filename,"has begun.")
    
    df, _, causes = data_opener.open_dataset_and_ground_truth(dataset, filename, rootdir=rootdir)
        
    df_train, df_test = separate_train_test(df)
    
    targets = df.columns
    
    for target in targets:
        if len(causes[target]) == 0:
            continue
        else:
            print("\tVariable", target, "is beginning.")
            
            time_algostart = time.time()
            lPred = bg.bivariate_granger(df_train, target, maxlags)
            time_algoend = time.time()
            
            #add autoregressive cause as the algorithm does not return it
            if target not in lPred:
                lPred.append(target)
            
            sPred = set(lPred)
            sTrue = set(causes[target])

            # metrics from results
            sTP = sTrue & sPred
            recall = len(sTP)/len(sTrue)
            precision = len(sTP)/len(sPred)
            
            base_row = {"dataset":dataset,"filename":filename,'target':target,
                        "totaltime":time_algoend - time_algostart,
                        "final_precision":precision,"final_recall":recall,
                        "final_TP":len(sTP), 
                        "final_FP":len(sPred)-len(sTP),
                        "final_FN": len(sTrue)-len(sTP), 
                        "final_TN": len(df.columns)+len(sTP)-len(sPred)-len(sTrue)}
            
            model = ARDLModel(model_config, target)
            df_test_bis = df_test.reset_index()  # Statsmodels has uncomprehensible behaviour
            df_train_bis = df_train.reset_index()  # Statsmodels has uncomprehensible behaviour
            model.fit(df_train_bis[lPred])
            test_statistics = model.statistics(df_test_bis[lPred])
            test_statistics = dict([("test_"+x,test_statistics[x]) for x in test_statistics])
            
            base_row = {**base_row, **test_statistics}
            
            df_results.append(base_row)
            
df = pd.DataFrame(df_results)
df["baseline"]="bg"
df["model"]="ARDL"

df.to_csv("baselines.csv",mode="a", index=False, header=False)

File random-rels_20_1C_returns30007000.csv has begun.
	Variable 2 is beginning.
	Variable 4 is beginning.
	Variable 5 is beginning.
	Variable 6 is beginning.
	Variable 7 is beginning.
	Variable 11 is beginning.
	Variable 13 is beginning.
	Variable 16 is beginning.
	Variable 17 is beginning.
	Variable 18 is beginning.
	Variable 22 is beginning.
	Variable 23 is beginning.
	Variable 24 is beginning.
File random-rels_20_1E_returns30007000.csv has begun.
	Variable 1 is beginning.
	Variable 3 is beginning.
	Variable 5 is beginning.
	Variable 9 is beginning.
	Variable 10 is beginning.
	Variable 12 is beginning.
	Variable 14 is beginning.
	Variable 16 is beginning.
	Variable 17 is beginning.
	Variable 19 is beginning.
	Variable 20 is beginning.
	Variable 22 is beginning.
	Variable 23 is beginning.
File random-rels_20_1_3_returns800012000.csv has begun.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 5 is beginning.
	Variable 6 is beginning.
	Variable 7 i

In [9]:
rootdir = "../"
dataset = "VARSmall/returns"

maxlags = 10
model_config = {
    "constructor":{
            "lags": maxlags,
            "order": maxlags,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

df_results = []

for filename in os.listdir(rootdir + "data/" + dataset + "/"):
    if not os.path.isfile(rootdir + "data/" + dataset + "/" + filename):
        continue
    print("File",filename,"has begun.")
    
    df, _, causes = data_opener.open_dataset_and_ground_truth(dataset, filename, rootdir=rootdir)
        
    df_train, df_test = separate_train_test(df)
    
    targets = df.columns
    
    for target in targets:
        if len(causes[target]) == 0:
            continue
        else:
            print("\tVariable", target, "is beginning.")
            
            time_algostart = time.time()
            lPred = bg.bivariate_granger(df_train, target, maxlags)
            time_algoend = time.time()
            
            #add autoregressive cause as the algorithm does not return it
            if target not in lPred:
                lPred.append(target)
            
            sPred = set(lPred)
            sTrue = set(causes[target])

            # metrics from results
            sTP = sTrue & sPred
            recall = len(sTP)/len(sTrue)
            precision = len(sTP)/len(sPred)
            
            base_row = {"dataset":dataset,"filename":filename,'target':target,
                        "totaltime":time_algoend - time_algostart,
                        "final_precision":precision,"final_recall":recall,
                        "final_TP":len(sTP), 
                        "final_FP":len(sPred)-len(sTP),
                        "final_FN": len(sTrue)-len(sTP), 
                        "final_TN": len(df.columns)+len(sTP)-len(sPred)-len(sTrue)}
            
            model = ARDLModel(model_config, target)
            df_test_bis = df_test.reset_index()  # Statsmodels has uncomprehensible behaviour
            df_train_bis = df_train.reset_index()  # Statsmodels has uncomprehensible behaviour
            model.fit(df_train_bis[lPred])
            test_statistics = model.statistics(df_test_bis[lPred])
            test_statistics = dict([("test_"+x,test_statistics[x]) for x in test_statistics])
            
            base_row = {**base_row, **test_statistics}
            
            df_results.append(base_row)
            
df = pd.DataFrame(df_results)
df["baseline"]="bg"
df["model"]="ARDL"

df.to_csv("baselines.csv",mode="a", index=False, header=False)

File data_1.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
	Variable 5 is beginning.
	Variable 6 is beginning.
	Variable 7 is beginning.
	Variable 8 is beginning.
	Variable 9 is beginning.
File data_0.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
	Variable 5 is beginning.
	Variable 6 is beginning.
	Variable 7 is beginning.
	Variable 8 is beginning.
	Variable 9 is beginning.
File data_2.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
	Variable 5 is beginning.
	Variable 6 is beginning.
	Variable 7 is beginning.
	Variable 8 is beginning.
	Variable 9 is beginning.


In [20]:
rootdir = "../"
dataset = "VARLarge/returns"

maxlags = 10
model_config = {
    "constructor":{
            "lags": maxlags,
            "order": maxlags,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

df_results = []

for filename in os.listdir(rootdir + "data/" + dataset + "/"):
    if not os.path.isfile(rootdir + "data/" + dataset + "/" + filename):
        continue
    print("File",filename,"has begun.")
    
    df, _, causes = data_opener.open_dataset_and_ground_truth(dataset, filename, rootdir=rootdir)
        
    df_train, df_test = separate_train_test(df)
    
    targets = df.columns
    
    for target in targets[:20]:
        if len(causes[target]) == 0:
            continue
        else:
            print("\tVariable", target, "is beginning.")
            
            time_algostart = time.time()
            lPred = bg.bivariate_granger(df_train, target, maxlags)
            time_algoend = time.time()
            
            #add autoregressive cause as the algorithm does not return it
            if target not in lPred:
                lPred.append(target)
            
            sPred = set(lPred)
            sTrue = set(causes[target])

            # metrics from results
            sTP = sTrue & sPred
            recall = len(sTP)/len(sTrue)
            precision = len(sTP)/len(sPred)
            
            base_row = {"dataset":dataset,"filename":filename,'target':target,
                        "totaltime":time_algoend - time_algostart,
                        "final_precision":precision,"final_recall":recall,
                        "final_TP":len(sTP), 
                        "final_FP":len(sPred)-len(sTP),
                        "final_FN": len(sTrue)-len(sTP), 
                        "final_TN": len(df.columns)+len(sTP)-len(sPred)-len(sTrue)}
            
            try:
                model = ARDLModel(model_config, target)
                df_test_bis = df_test.reset_index()  # Statsmodels has uncomprehensible behaviour
                df_train_bis = df_train.reset_index()  # Statsmodels has uncomprehensible behaviour
                model.fit(df_train_bis[lPred])
                test_statistics = model.statistics(df_test_bis[lPred])
                test_statistics = dict([("test_"+x,test_statistics[x]) for x in test_statistics])
            
                base_row = {**base_row, **test_statistics}
            except:
                1
            
            df_results.append(base_row)
            
df = pd.DataFrame(df_results)
df["baseline"]="bg"
df["model"]="ARDL"

ref = pd.read_csv('baselines.csv')
df = pd.concat([ref, df])

df.to_csv("baselines.csv", index=False)

File data_0.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
	Variable 5 is beginning.
	Variable 6 is beginning.
	Variable 7 is beginning.
	Variable 8 is beginning.
	Variable 9 is beginning.
	Variable 10 is beginning.
	Variable 11 is beginning.
	Variable 12 is beginning.
	Variable 13 is beginning.
	Variable 14 is beginning.
	Variable 15 is beginning.
	Variable 16 is beginning.
	Variable 17 is beginning.
	Variable 18 is beginning.
	Variable 19 is beginning.


# TFT

In [3]:
def main_tft(dataset, maxlags, n_epochs, rootdir="../"):
    config = {"input_chunk_length":maxlags,
              "n_epochs":n_epochs}
    df_results = []

    for filename in os.listdir(rootdir + "data/" + dataset + "/"):
        if not os.path.isfile(rootdir + "data/" + dataset + "/" + filename):
            continue
        print("File",filename,"has begun.")

        df, _, causes = data_opener.open_dataset_and_ground_truth(dataset, filename, rootdir=rootdir)

        df_train, df_test = separate_train_test(df)

        targets = df.columns

        for target in targets:
            if len(causes[target]) == 0:
                continue
            else:
                print("\tVariable", target, "is beginning.")

                time_algostart = time.time()
                test_statistics = TFT_baseline(df_train, df_test, target, config)
                time_algoend = time.time()


                base_row = {"dataset":dataset,"filename":filename,'target':target,
                            "totaltime":time_algoend - time_algostart}

                test_statistics = dict([("test_"+x,test_statistics[x]) for x in test_statistics])

                base_row = {**base_row, **test_statistics}

                df_results.append(base_row)
    return df_results

In [5]:
rootdir = "../"
dataset = "SynthNonlin/7ts2h"

maxlags = 10
n_epochs = 10


df_results = main_tft(dataset, maxlags, n_epochs)
            

df = pd.DataFrame(df_results)
df["baseline"]="TFT"
df["model"]="TFT"

ref = pd.read_csv('baselines.csv')
df = pd.concat([ref, df])

df.to_csv("baselines.csv", index=False)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


File data_1.csv has begun.
	Variable E is beginning.


`Trainer.fit` stopped: `max_epochs=10` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more d

	Variable B is beginning.


`Trainer.fit` stopped: `max_epochs=10` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more d

...

`Trainer.fit` stopped: `max_epochs=10` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more d

	Variable D is beginning.


`Trainer.fit` stopped: `max_epochs=10` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more d

	Variable A is beginning.


`Trainer.fit` stopped: `max_epochs=10` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [6]:
rootdir = "../"
dataset = "fMRI_processed_by_Nauta/returns/our_selection"

maxlags = 5
n_epochs = 10


df_results = main_tft(dataset, maxlags, n_epochs)
            

df = pd.DataFrame(df_results)
df["baseline"]="TFT"
df["model"]="TFT"

ref = pd.read_csv('baselines.csv')
df = pd.concat([ref, df])

df.to_csv("baselines.csv", index=False)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


File timeseries21.csv has begun.
	Variable 0 is beginning.


`Trainer.fit` stopped: `max_epochs=10` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more d

	Variable 1 is beginning.


`Trainer.fit` stopped: `max_epochs=10` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more d

...

`Trainer.fit` stopped: `max_epochs=10` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more d

	Variable 3 is beginning.


`Trainer.fit` stopped: `max_epochs=10` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more d

	Variable 4 is beginning.


`Trainer.fit` stopped: `max_epochs=10` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [7]:
rootdir = "../"
dataset = "TestCLIM_N-5_T-250/returns"

maxlags = 5
n_epochs = 10


df_results = main_tft(dataset, maxlags, n_epochs)
            

df = pd.DataFrame(df_results)
df["baseline"]="TFT"
df["model"]="TFT"

ref = pd.read_csv('baselines.csv')
df = pd.concat([ref, df])

df.to_csv("baselines.csv", index=False)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


File TestCLIM_N-5_T-250_0089.txt has begun.
	Variable 0 is beginning.


`Trainer.fit` stopped: `max_epochs=10` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more d

	Variable 1 is beginning.


`Trainer.fit` stopped: `max_epochs=10` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more d

...

`Trainer.fit` stopped: `max_epochs=10` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more d

	Variable 3 is beginning.


`Trainer.fit` stopped: `max_epochs=10` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more d

	Variable 4 is beginning.


`Trainer.fit` stopped: `max_epochs=10` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [8]:
rootdir = "../"
dataset = "VARSmall/returns"

maxlags = 10
n_epochs = 10


df_results = main_tft(dataset, maxlags, n_epochs)
            

df = pd.DataFrame(df_results)
df["baseline"]="TFT"
df["model"]="TFT"

ref = pd.read_csv('baselines.csv')
df = pd.concat([ref, df])

df.to_csv("baselines.csv", index=False)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


File data_1.csv has begun.
	Variable 0 is beginning.


`Trainer.fit` stopped: `max_epochs=10` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more d

	Variable 1 is beginning.


`Trainer.fit` stopped: `max_epochs=10` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more d

...

`Trainer.fit` stopped: `max_epochs=10` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more d

	Variable 8 is beginning.


`Trainer.fit` stopped: `max_epochs=10` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more d

	Variable 9 is beginning.


`Trainer.fit` stopped: `max_epochs=10` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [9]:
rootdir = "../"
dataset = "FinanceCPT/returns/our_selection"

maxlags = 10
n_epochs = 10


df_results = main_tft(dataset, maxlags, n_epochs)
            

df = pd.DataFrame(df_results)
df["baseline"]="TFT"
df["model"]="TFT"

ref = pd.read_csv('baselines.csv')
df = pd.concat([ref, df])

df.to_csv("baselines.csv", index=False)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


File random-rels_20_1C_returns30007000.csv has begun.
	Variable 2 is beginning.


`Trainer.fit` stopped: `max_epochs=10` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more d

	Variable 4 is beginning.


`Trainer.fit` stopped: `max_epochs=10` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more d

...

`Trainer.fit` stopped: `max_epochs=10` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more d

	Variable 23 is beginning.


`Trainer.fit` stopped: `max_epochs=10` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more d

	Variable 24 is beginning.


`Trainer.fit` stopped: `max_epochs=10` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


# RFE - with ARDL model evaluator

In [12]:
def main_rfe(dataset, rootdir, maxlags):
    model_config = {
        "constructor":{
                "lags": maxlags,
                "order": maxlags,
                "causal": True,
                "trend": "ct",
                "seasonal": False,
                "period": None,
                "missing": "drop"
            },
            "fit":{
                "cov_type": "HC0"
            }
        }

    df_results = []

    for filename in os.listdir(rootdir + "data/" + dataset + "/"):
        if not os.path.isfile(rootdir + "data/" + dataset + "/" + filename):
            continue
        print("File",filename,"has begun.")

        df, _, causes = data_opener.open_dataset_and_ground_truth(dataset, filename, rootdir=rootdir)

        df_train, df_test = separate_train_test(df)

        targets = df.columns

        for target in targets:
            if len(causes[target]) == 0:
                continue
            else:
                print("\tVariable", target, "is beginning.")

                time_algostart = time.time()
                lPred, ranking = rfe.get_features(df_train, target, maxlags, len(causes[target]))
                time_algoend = time.time()

                #add autoregressive cause as the algorithm does not return it
                if target not in lPred:
                    lPred = list(lPred)
                    lPred.append(target)

                sPred = set(lPred)
                sTrue = set(causes[target])

                # metrics from results
                sTP = sTrue & sPred
                recall = len(sTP)/len(sTrue)
                precision = len(sTP)/len(sPred)

                base_row = {"dataset":dataset,"filename":filename,'target':target,
                            "totaltime":time_algoend - time_algostart,
                            "final_precision":precision,"final_recall":recall,
                            "final_TP":len(sTP), 
                            "final_FP":len(sPred)-len(sTP),
                            "final_FN": len(sTrue)-len(sTP), 
                            "final_TN": len(df.columns)+len(sTP)-len(sPred)-len(sTrue)}

                model = ARDLModel(model_config, target)
                df_test_bis = df_test.reset_index(drop=True)  # Statsmodels has uncomprehensible behaviour
                df_train_bis = df_train.reset_index(drop=True)  # Statsmodels has uncomprehensible behaviour
                model.fit(df_train_bis[lPred])
                test_statistics = model.statistics(df_test_bis[lPred])
                test_statistics = dict([("test_"+x,test_statistics[x]) for x in test_statistics])

                base_row = {**base_row, **test_statistics}

                df_results.append(base_row)
    return df_results

In [13]:
rootdir = "../"
dataset = "SynthNonlin/7ts2h"


maxlags = 10

df_results = main_rfe(dataset, rootdir, maxlags)
            
df = pd.DataFrame(df_results)
df["baseline"]="rfe"
df["model"]="ARDL"

ref = pd.read_csv('baselines.csv')
df = pd.concat([ref, df])

df.to_csv("baselines.csv", index=False)

File data_1.csv has begun.
	Variable E is beginning.
	Variable B is beginning.
	Variable F is beginning.
	Variable C is beginning.
	Variable H is beginning.
	Variable D is beginning.
	Variable A is beginning.
File data_0.csv has begun.
	Variable E is beginning.
	Variable B is beginning.
	Variable F is beginning.
	Variable C is beginning.
	Variable H is beginning.
	Variable D is beginning.
	Variable A is beginning.
File data_2.csv has begun.
	Variable E is beginning.
	Variable B is beginning.
	Variable F is beginning.
	Variable C is beginning.
	Variable H is beginning.
	Variable D is beginning.
	Variable A is beginning.
File data_6.csv has begun.
	Variable E is beginning.
	Variable B is beginning.
	Variable F is beginning.
	Variable C is beginning.
	Variable H is beginning.
	Variable D is beginning.
	Variable A is beginning.
File data_9.csv has begun.
	Variable E is beginning.
	Variable B is beginning.
	Variable F is beginning.
	Variable C is beginning.
	Variable H is beginning.
	Variab

In [14]:
rootdir = "../"
dataset = "fMRI_processed_by_Nauta/returns/our_selection"


maxlags = 5

df_results = main_rfe(dataset, rootdir, maxlags)
            
df = pd.DataFrame(df_results)
df["baseline"]="rfe"
df["model"]="ARDL"

ref = pd.read_csv('baselines.csv')
df = pd.concat([ref, df])

df.to_csv("baselines.csv", index=False)

File timeseries21.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries7.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries23.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries15.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries16.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries18.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries8.csv ha

In [15]:
rootdir = "../"
dataset = "TestCLIM_N-5_T-250/returns"


maxlags = 5

df_results = main_rfe(dataset, rootdir, maxlags)
            
df = pd.DataFrame(df_results)
df["baseline"]="rfe"
df["model"]="ARDL"

ref = pd.read_csv('baselines.csv')
df = pd.concat([ref, df])

df.to_csv("baselines.csv", index=False)

File TestCLIM_N-5_T-250_0089.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0074.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0149.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0055.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0154.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0110.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variabl

	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0072.txt has begun.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0075.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0008.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0081.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0052.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0006.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variabl

File TestCLIM_N-5_T-250_0167.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0080.txt has begun.
	Variable 0 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0077.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0078.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0073.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0122.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File Tes

	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0046.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0005.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0178.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0064.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0158.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_01

In [16]:
rootdir = "../"
dataset = "VARSmall/returns"


maxlags = 10

df_results = main_rfe(dataset, rootdir, maxlags)
            
df = pd.DataFrame(df_results)
df["baseline"]="rfe"
df["model"]="ARDL"

ref = pd.read_csv('baselines.csv')
df = pd.concat([ref, df])

df.to_csv("baselines.csv", index=False)

File data_1.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
	Variable 5 is beginning.
	Variable 6 is beginning.
	Variable 7 is beginning.
	Variable 8 is beginning.
	Variable 9 is beginning.
File data_0.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
	Variable 5 is beginning.
	Variable 6 is beginning.
	Variable 7 is beginning.
	Variable 8 is beginning.
	Variable 9 is beginning.
File data_2.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
	Variable 5 is beginning.
	Variable 6 is beginning.
	Variable 7 is beginning.
	Variable 8 is beginning.
	Variable 9 is beginning.


In [17]:
rootdir = "../"
dataset = "FinanceCPT/returns/our_selection"


maxlags = 10

df_results = main_rfe(dataset, rootdir, maxlags)
            
df = pd.DataFrame(df_results)
df["baseline"]="rfe"
df["model"]="ARDL"

ref = pd.read_csv('baselines.csv')
df = pd.concat([ref, df])

df.to_csv("baselines.csv", index=False)

File random-rels_20_1C_returns30007000.csv has begun.
	Variable 2 is beginning.
	Variable 4 is beginning.
	Variable 5 is beginning.
	Variable 6 is beginning.
	Variable 7 is beginning.
	Variable 11 is beginning.
	Variable 13 is beginning.
	Variable 16 is beginning.
	Variable 17 is beginning.
	Variable 18 is beginning.
	Variable 22 is beginning.
	Variable 23 is beginning.
	Variable 24 is beginning.
File random-rels_20_1E_returns30007000.csv has begun.
	Variable 1 is beginning.
	Variable 3 is beginning.
	Variable 5 is beginning.
	Variable 9 is beginning.
	Variable 10 is beginning.
	Variable 12 is beginning.
	Variable 14 is beginning.
	Variable 16 is beginning.
	Variable 17 is beginning.
	Variable 19 is beginning.
	Variable 20 is beginning.
	Variable 22 is beginning.
	Variable 23 is beginning.
File random-rels_20_1_3_returns800012000.csv has begun.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 5 is beginning.
	Variable 6 is beginning.
	Variable 7 i